# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.12.45 llama-index-finetuning==0.3.2 llama-index-embeddings-adapter==0.3.0 openai==1.93.0 tiktoken==0.9.0 chromadb==1.0.15 llama-index-vector-stores-chroma==0.4.2 \
                html2text==2024.2.26 llama-index-llms-openai==0.4.5 llama-index-embeddings-huggingface==0.5.5 llama-index-embeddings-openai==0.3.1 llama-index-llms-azure-openai==0.3.4 jedi==0.19.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Download the Dataset


In [3]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

ai_tutor_knowledge.jsonl: 0.00B [00:00, ?B/s]

In [4]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

## LlamaIndex Document

In [5]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

### Splitting Dataset


In [6]:
import random

random.shuffle(doc)
split_index = int(len(doc) * 0.9)

# TRAIN_DOCs and VALIDATION_DOCs
TRAIN_DOCs = doc[:split_index]
VALIDATION_DOCs = doc[split_index:]

# Chunking


In [7]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import Document

# Now use the parser
parser = SimpleNodeParser.from_defaults(chunk_size=768, chunk_overlap=64)
TRAIN_NODEs = parser.get_nodes_from_documents(TRAIN_DOCs)
VALIDATION_NODEs = parser.get_nodes_from_documents(VALIDATION_DOCs)

print(len(TRAIN_NODEs), len(VALIDATION_NODEs))

2817 297


In [8]:
# Use a subset of the dataset if testing.

# Test with a few sample, processing dataset fully can be costly depanding on the size.
# NOTE: Checkpoints are provided in the lesson, so no need to run the code on full dataset.

testing =True

if testing:
    TRAIN_NODEs = TRAIN_NODEs[0:10]
    VALIDATION_NODEs = VALIDATION_NODEs[0:5]

# Generate Question


We use a Large Language Model (LLM) to produce questions for each chunk of the dataset. Then we can use these data to train the model to develop embeddings that more accurately represent the types of questions users may ask.


In [9]:
# Use this block of code if you don't want to generate the questions for the dataset. (Avoid API call charges!)
# Uncomment the following code, and keep in mind to comment all the contents in the next coding block.


from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini", temperature=1, max_tokens=512)

# Generate questions for each chunk.

TRAIN_DATASET = generate_qa_embedding_pairs(TRAIN_NODEs, llm=llm, output_path="./train_dataset.json")

VALIDATION_DATASET = generate_qa_embedding_pairs(VALIDATION_NODEs, llm=llm, output_path="./val_dataset.json")

100%|██████████| 10/10 [00:18<00:00,  1.86s/it]


Final dataset saved.


100%|██████████| 5/5 [00:20<00:00,  4.02s/it]

Final dataset saved.


In [10]:
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="jaiganesan/Embedding-model-fine-tuning-dataset", repo_type="dataset",local_dir="/content/")


# from llama_index.finetuning import EmbeddingQAFinetuneDataset

# # Load the pre-generated questions json files.
# TRAIN_DATASET = EmbeddingQAFinetuneDataset.from_json("./train_dataset.json")
# VALIDATION_DATASET = EmbeddingQAFinetuneDataset.from_json("./val_dataset.json")

# Load an Embedding Model


In [11]:
from llama_index.core.embeddings import resolve_embed_model

# Load an existing embedding model with a adapter layer on top.
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
import torch

# Finetune the adapter
finetune_engine = EmbeddingAdapterFinetuneEngine(
    TRAIN_DATASET,
    base_embed_model,
    model_output_path="model_output_test",
    epochs=2,
    verbose=True,
    bias=True,
)

In [13]:
# Initiate the Finetuning process
finetune_engine.finetune()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 1.1736323833465576
> [Epoch 0] Current loss: 0.9480637311935425


Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 1.166783094406128
> [Epoch 1] Current loss: 0.9456075429916382
> Finished training, saving to model_output_test


In [14]:
embed_model = finetune_engine.get_finetuned_model()

# Or, import model from the directory whenever required.
# from llama_index.core.embeddings import LinearAdapterEmbeddingModel
# embed_model = LinearAdapterEmbeddingModel(base_embed_model, "model_output_test")

In [15]:
embed_model

AdapterEmbeddingModel(model_name='Adapter for BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7b698b5f3010>, num_workers=None, embeddings_cache=None)

## Fine tuning OpenAI Embedding Model using Adapter method

In [16]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.embeddings.openai import OpenAIEmbedding

openai_finetune_engine = EmbeddingAdapterFinetuneEngine(
    TRAIN_DATASET,
    OpenAIEmbedding(model="text-embedding-3-small"),
    model_output_path="model_output_test_openai",
    bias=True,
    epochs=2,
    verbose=True,
)

In [17]:
openai_finetune_engine.finetune()

openai_embed_model = openai_finetune_engine.get_finetuned_model()

> Prepared optimizer, scheduler, and loss model.


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

> [Epoch 0] Current loss: 0.8366237878799438
> [Epoch 0] Current loss: 0.8464261293411255


Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

> [Epoch 1] Current loss: 0.8210369348526001
> [Epoch 1] Current loss: 0.8273979425430298
> Finished training, saving to model_output_test_openai


In [18]:
openai_embed_model

AdapterEmbeddingModel(model_name='Adapter for text-embedding-3-small', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7b698a7dc1d0>, num_workers=None, embeddings_cache=None)

# Evaluate


## Define the Evaluation Functions


In [19]:
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from llama_index.core import Settings
from tqdm import tqdm
import pandas as pd

def evaluate(dataset, embedding_model, top_k=5, verbose=False):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    Settings.embed_model = embedding_model

    # Chunking the documents and generating embeddings
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(nodes, show_progress=True)

    # Define a retriever to answer the questions
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []

    # Look into each response sources to see if the chunk that contains the answer is retrieved.
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]

        try:
            rank = retrieved_ids.index(expected_id) + 1
            reciprocal_rank = 1 / rank
        except ValueError:
            rank = None
            reciprocal_rank = 0

        is_hit = expected_id in retrieved_ids

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
            "rank": rank,
            "reciprocal_rank": reciprocal_rank
        }
        eval_results.append(eval_result)

    return eval_results

## OpenAI Embedding Model Evaluation


In [20]:
# Load the OpenAI Ada model and evaluate it.
openai_text_embedding_small = OpenAIEmbedding(model="text-embedding-3-small")
openai_embedding_val_results = evaluate(VALIDATION_DATASET, embedding_model=openai_text_embedding_small)

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:01<00:00,  5.27it/s]


In [21]:
openai_embedding_val_results = [
    result for result in openai_embedding_val_results if isinstance(result, dict)
]

df_openai = pd.DataFrame(openai_embedding_val_results)

hit_rate_openai = df_openai["is_hit"].mean()
mrr_openai = df_openai["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_openai}")
print(f"MRR: {mrr_openai}")

Hit rate: 1.0
MRR: 0.8333333333333333


### OpenAI Embedding Model with Fine Tuned Adapter Model Evaluation

In [22]:
from llama_index.embeddings.adapter import AdapterEmbeddingModel

openai_embed_model = AdapterEmbeddingModel(openai_text_embedding_small, "model_output_test_openai")

val_results_ft_openai = evaluate(VALIDATION_DATASET, embedding_model = openai_embed_model)

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:01<00:00,  5.86it/s]


In [23]:
val_results_ft_openai = [
    result for result in val_results_ft_openai if isinstance(result, dict)
]

df_openai_ft = pd.DataFrame(val_results_ft_openai)

hit_rate_openai_ft = df_openai_ft["is_hit"].mean()
mrr_openai_ft = df_openai_ft["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_openai_ft}")
print(f"MRR: {mrr_openai_ft}")

Hit rate: 1.0
MRR: 0.8333333333333333


## Open Source BAAI Model Evaluation


In [24]:
# Load the Base model without fine-tuning
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
bge_val_results = evaluate(VALIDATION_DATASET, embedding_model=base_embed_model)

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 15.94it/s]


In [25]:
bge_val_results = [
    result for result in bge_val_results if isinstance(result, dict)
]

df_bge = pd.DataFrame(bge_val_results)

hit_rate_bge = df_bge["is_hit"].mean()
mrr_bge = df_bge["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_bge}")
print(f"MRR: {mrr_bge}")

Hit rate: 1.0
MRR: 0.7499999999999999


## FineTuned BAAI Adapter Embedding Model Evaluation


In [26]:
from llama_index.embeddings.adapter import AdapterEmbeddingModel

# Load the Fine-tuned model.
embed_model = AdapterEmbeddingModel(base_embed_model, "model_output_test")

val_results_finetuned = evaluate(VALIDATION_DATASET, embedding_model=embed_model)

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 14.32it/s]


In [27]:
val_results_finetuned = [
    result for result in val_results_finetuned if isinstance(result, dict)
]

df_finetuned = pd.DataFrame(val_results_finetuned)

hit_rate_finetuned = df_finetuned["is_hit"].mean()
mrr_finetuned = df_finetuned["reciprocal_rank"].mean()

print(f"Hit rate: {hit_rate_finetuned}")
print(f"MRR: {mrr_finetuned}")

Hit rate: 1.0
MRR: 0.7499999999999999
